# 🏃 Strava Activity Dashboard

This notebook provides an interactive dashboard for your Strava activities using Pandas and Plotly.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown
import sys
from pathlib import Path

# Add src to path
sys.path.append(str(Path.cwd() / 'src'))

from src.data_manager import DataManager
from src.data_processing import ActivityProcessor

# Load Data
try:
    dm = DataManager()
    if not dm.activities_csv.exists() and not dm.activities_json.exists():
        print("⚠️ No data found! Please run 'python -m src.data_manager' in your terminal first.")
    else:
        df = dm.load_activities_as_dataframe()
        # Ensure datetime
        if 'start_date_local' in df.columns:
            df['start_date_local'] = pd.to_datetime(df['start_date_local'])
        
        print(f"✅ Successfully loaded {len(df)} activities.")
        
        # Initialize Processor
        processor = ActivityProcessor(df)
except Exception as e:
    print(f"❌ Error loading data: {e}")

## 📊 Global Statistics

In [ ]:
if 'processor' in locals():
    stats = processor.get_summary_stats()
    
    # Create a nice summary dataframe
    summary_data = {
        'Metric': ['Total Activities', 'Total Distance', 'Total Elevation', 'Moving Time', 'Avg Speed'],
        'Value': [
            f"{stats['total_activities']}",
            f"{stats['total_distance_km']:.1f} km",
            f"{stats['total_elevation_gain_m']:.0f} m",
            f"{stats['total_moving_time_hours']:.1f} hrs",
            f"{stats['average_speed_kmh']:.1f} km/h"
        ]
    }
    
    display(pd.DataFrame(summary_data).style.hide(axis='index'))

## 📈 Visualizations

In [ ]:
if 'processor' in locals():
    # 1. Activities by Type (Donut Chart)
    if 'activity_types' in stats and stats['activity_types']:
        type_df = pd.DataFrame(list(stats['activity_types'].items()), columns=['Type', 'Count'])
        
        fig_pie = px.pie(
            type_df, 
            values='Count', 
            names='Type', 
            title='Activities by Type',
            hole=0.4,
            color_discrete_sequence=px.colors.qualitative.Vivid
        )
        fig_pie.show()
    
    # 2. Weekly Distance (Bar Chart)
    weekly_df = processor.get_weekly_aggregates().reset_index()
    if not weekly_df.empty:
        # Create date for x-axis
        weekly_df['week_start'] = weekly_df.apply(
            lambda row: pd.to_datetime(f"{int(row['year'])}-W{int(row['week'])}-1", format="%G-W%V-%u"), 
            axis=1
        )
        
        fig_bar = px.bar(
            weekly_df, 
            x='week_start', 
            y='distance_km', 
            title='Weekly Distance Progress',
            labels={'week_start': 'Week', 'distance_km': 'Distance (km)'}
        )
        fig_bar.show()

## 📝 Recent Activities

In [ ]:
if 'df' in locals():
    display_cols = [
        'start_date_local', 'name', 'type', 'distance_km', 
        'moving_time_min', 'total_elevation_gain', 'average_speed_kmh'
    ]
    
    # Filter existing columns only
    valid_cols = [c for c in display_cols if c in df.columns]
    
    recent_df = df[valid_cols].sort_values('start_date_local', ascending=False).head(10)
    
    # Rename for display
    recent_df = recent_df.rename(columns={
        'start_date_local': 'Date',
        'name': 'Activity Name',
        'type': 'Type',
        'distance_km': 'Dist (km)',
        'moving_time_min': 'Time (min)',
        'total_elevation_gain': 'Elev (m)',
        'average_speed_kmh': 'Speed (km/h)'
    })
    
    display(recent_df)